In [18]:
import random
import os
import shutil
import glob
import gc
import sys
import h5py
import time
import datetime
import pickle
import librosa
import warnings
import matplotlib.pyplot as plt

import numpy as np
from numpy import array
import pandas as pd
from pandas.plotting import parallel_coordinates
from pydub import AudioSegment

#imported for testing
import wave
import contextlib

# for outputing file
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.stats.stats as st

from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler

import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential, save_model
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical
from keras import metrics

from colorama import Fore, Back, Style

from IPython.display import clear_output


#warnings.filterwarnings('ignore')
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU
from keras import backend
# print(len(backend.tensorflow_backend._get_available_gpus()) > 0)

#warnings.filterwarnings('ignore')
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

sys.path.insert(1, '..//components//')
import load_feat_directories

In [19]:
def load_X(x_directory, label):
    X = np.array([])
    Y = []
    for file in os.listdir(x_directory):
        if file.endswith('.npy'):
            x = np.load(x_directory + file)
            x = np.array([list(x)])
            if len(X) == 0:
                X = x
            else:
                X = np.vstack((X,x))
            
            Y.append(label)

    return X, Y

In [20]:
CaFE_happy = 'D://Datasets//CaFE//npy//Happy//two//'
CaFE_angry = 'D://Datasets//CaFE//npy//Angry//two//'
CaFE_neutral = 'D://Datasets//CaFE//npy//Neutral//two//'
CaFE_sad = 'D://Datasets//CaFE//npy//Sad//two//'

CaFE_npy = [CaFE_happy, CaFE_angry, CaFE_neutral, CaFE_sad]

In [22]:
X_happy, Y_happy = load_X(CaFE_npy[0], 0)
X_angry, Y_angry = load_X(CaFE_npy[1], 0)
X_neutral, Y_neutral = load_X(CaFE_npy[2], 1)
X_sad, Y_sad = load_X(CaFE_npy[3], 1)

Xs = [X_happy, X_angry, X_neutral, X_sad]
Ys = [Y_happy, Y_angry, Y_neutral, Y_sad]

X = []
Y = []

for item in Xs:
    if len(X) == 0: X = item
    else: X = np.vstack((X,item))

for item in Ys:
    Y = Y + item
Label = to_categorical(np.array(Y))

scaler = StandardScaler()
print(scaler.fit(X))
featureSet = scaler.transform(X)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [23]:
print(Label)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [46]:
adam = optimizers.Adam(lr = 5e-3, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0, amsgrad = True)
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
rmsprop = optimizers.RMSprop(lr = 0.0001, rho = 0.9, epsilon = None, decay = 0.0)
adagrad = optimizers.Adagrad(lr = 0.01, epsilon = None, decay = 0.0)
adadelta = optimizers.Adadelta(lr = 1.0, rho = 0.95, epsilon = None, decay = 0.0)
adamax = optimizers.Adamax(lr = 0.002, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0)
nadam = optimizers.Nadam(lr = 0.002, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, schedule_decay = 0.004)

In [47]:
def wall(block):
    wall = np.zeros((10,10),dtype=np.float)
    #block = np.arange(1,7).reshape(2,3)

    x = 2
    y = 3
    wall[x:x+block.shape[0], y:y+block.shape[1]] = block
    
    return wall

In [48]:
def create_cnn(title, num_layers, n_neurons, n_batch, nbindex, dropout, classes, dense_layers):

    model = Sequential()

    #model.add(Convolution1D(nb_filter=nbindex, filter_length=fillength, activation = 'relu', input_shape=(1, 2), kernel_constraint=maxnorm(2)))
    #model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
    #model.add(Dropout(dropout))
    #model.add(Flatten())
    
    for i in range(0, dense_layers):
        model.add(Dense(n_neurons, input_dim=2, activation='relu'))
        model.add(Dropout(dropout))
    
    model.add(Dense(classes, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer=adam,
                  metrics=[metrics.categorical_accuracy])

    model.summary()

    return model

In [51]:
def train_cnn(prefix):
    
    save_to_path = prefix

    checkpoint_filepath = prefix + "Checkpoint_" + title + ".hdf5"
    final_filepath = prefix + "Final_" + title + ".hdf5"

    if not os.path.exists(save_to_path):
        os.mkdir(save_to_path)

    X, X_test, Y, Y_test= train_test_split(featureSet, Label, test_size = 0.25, shuffle = True)
    model = create_cnn(title, num_layers, n_neurons, n_batch, nbindex, dropout, classes, dense_layers)
    checkpoint = ModelCheckpoint(checkpoint_filepath, monitor = 'val_categorical_accuracy', verbose = 1, save_best_only = True, mode = 'auto')
    early_stopping_monitor = EarlyStopping(patience = 10)
    callbacks_list = [checkpoint, early_stopping_monitor]
    model.fit(X, Y, nb_epoch = n_epoch, batch_size = n_batch,  callbacks = callbacks_list, validation_data = (X_test, Y_test), verbose = 1)
    model.save(final_filepath)
    return model

In [55]:
classes = 2
n_neurons = 32
dense_layers = 1
num_layers = 1
fillength = 1
nbindex = 256
dropout = 0.2
n_batch = 32
n_epoch = 1000
title = 'Top_neurons_' + str(n_neurons) + '_filters_' + str(nbindex) + '_dropout_' + str(dropout) + '_epoch_' + str(n_epoch)
prefix = '..//..//modules//tf//'
final_filepath = prefix + "Final_" + title + ".hdf5"

model = train_cnn(prefix)


Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 32)                96        
_________________________________________________________________
dropout_32 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 2)                 66        
Total params: 162
Trainable params: 162
Non-trainable params: 0
_________________________________________________________________


C:\Users\yg9ca\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  app.launch_new_instance()


Train on 1296 samples, validate on 432 samples
Epoch 1/1000
1296/1296 [==============================] - 1s 617us/step - loss: 0.6696 - categorical_accuracy: 0.6003 - val_loss: 0.6593 - val_categorical_accuracy: 0.6389

Epoch 00001: val_categorical_accuracy improved from -inf to 0.63889, saving model to ..//..//modules//tf//Checkpoint_Top_neurons_32_filters_256_dropout_0.2_epoch_1000.hdf5
Epoch 2/1000
1296/1296 [==============================] - 0s 47us/step - loss: 0.6726 - categorical_accuracy: 0.6034 - val_loss: 0.6447 - val_categorical_accuracy: 0.6435

Epoch 00002: val_categorical_accuracy improved from 0.63889 to 0.64352, saving model to ..//..//modules//tf//Checkpoint_Top_neurons_32_filters_256_dropout_0.2_epoch_1000.hdf5
Epoch 3/1000
1296/1296 [==============================] - 0s 49us/step - loss: 0.6672 - categorical_accuracy: 0.6165 - val_loss: 0.6429 - val_categorical_accuracy: 0.6435

Epoch 00003: val_categorical_accuracy did not improve from 0.64352
Epoch 4/1000
1296/1296